In [104]:
from keras.models import Sequential
from keras.layers import MaxPooling2D, Conv2D, BatchNormalization, Flatten
import numpy as np
import tensorflow as tf
import pandas as pd

In [118]:
data = pd.read_csv('.\\data\\digit-recognizer\\train.csv')
split_ratio = 0.8
total_size = len(data)
train_size = int(split_ratio * total_size)
train_data = data[:train_size]
test_data = data[train_size:]
print(len(train_data))
print(len(test_data))

33600
8400


In [119]:
train_data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [120]:
y_train = train_data['label']
x_train = train_data.drop(['label'], axis = 1)

In [121]:
x_train.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [122]:
y_train.head()

0    1
1    0
2    1
3    4
4    0
Name: label, dtype: int64

In [123]:
print(len(x_train))

33600


In [124]:
batch_size = 100
epochs = 3

In [125]:
def get_batched_data():
    current_pos = 0
     
    total_size = len(x_train)
    steps_per_epoch = total_size / batch_size
    
    def one_hot_encode_y(y_value):
        ohe_vector = np.zeros((10, ))
        ohe_vector[y_value] = 1
        return ohe_vector
    
    for i in np.arange(steps_per_epoch):
        x_batched = x_train[current_pos: current_pos + batch_size].values
        x_batched_reshaped = np.array([x.reshape((28, 28, 1)) for x  in x_batched])#list(map(lambda x: x.reshape((28, 28)), x_batched)))
        y_batched = y_train[current_pos: current_pos + batch_size].values.tolist()
        y_batched_encoded = np.array([one_hot_encode_y(y) for y in y_batched])#list(map(lambda x: one_hot_encode_y(x), y_batched)))
        current_pos += batch_size
        yield x_batched_reshaped, y_batched_encoded

In [127]:
def build_model():
    model = Sequential()    
    model.add(Conv2D(32, 3, input_shape = (28, 28, 1), strides = (1, 1), padding = 'valid'))
    model.add(MaxPooling2D(pool_size = (2, 2), strides = None, padding = 'valid'))
    model.add(Conv2D(64, 3, strides = (1, 1), padding = 'valid'))
    model.add(MaxPooling2D(pool_size = (2, 2), strides = None, padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, 3, strides = (1, 1), padding = 'valid'))
    model.add(MaxPooling2D(pool_size = (2, 2), strides = None, padding = 'valid'))
    model.add(Flatten())
    model.add(Dense(10, activation = 'softmax'))
    return model 

In [128]:
model = build_model()
model.compile(optimizer = 'Adam',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

for epoch in np.arange(epochs):
    step_count = 0
    for x_batched, y_batched in get_batched_data():
        loss, accuracy = model.train_on_batch(x_batched, y_batched)

        if step_count % 10 == 0:
            print('Epoch: {}, step_count: {}, loss: {}, accuracy: {}'.format(epoch, step_count, loss, accuracy))

        step_count += 1

Epoch: 0, step_count: 0, loss: 2.962376594543457, accuracy: 0.20000000298023224
Epoch: 0, step_count: 10, loss: 0.6577691435813904, accuracy: 0.8100000023841858
Epoch: 0, step_count: 20, loss: 0.45939865708351135, accuracy: 0.8899999856948853
Epoch: 0, step_count: 30, loss: 0.40545177459716797, accuracy: 0.8600000143051147
Epoch: 0, step_count: 40, loss: 0.2637174129486084, accuracy: 0.8999999761581421
Epoch: 0, step_count: 50, loss: 0.2307525873184204, accuracy: 0.9200000166893005
Epoch: 0, step_count: 60, loss: 0.12338673323392868, accuracy: 0.9800000190734863
Epoch: 0, step_count: 70, loss: 0.17092952132225037, accuracy: 0.9399999976158142
Epoch: 0, step_count: 80, loss: 0.1651855707168579, accuracy: 0.9399999976158142
Epoch: 0, step_count: 90, loss: 0.26649561524391174, accuracy: 0.9100000262260437
Epoch: 0, step_count: 100, loss: 0.09686088562011719, accuracy: 0.9800000190734863
Epoch: 0, step_count: 110, loss: 0.1924799382686615, accuracy: 0.9399999976158142
Epoch: 0, step_count:

In [129]:
y_test = test_data['label']
x_test = test_data.drop(['label'], axis = 1)

In [141]:
def get_test_data():
    
    x_batched = x_test.values
    x_batched_reshaped = np.array([x.reshape((28, 28, 1)) for x  in x_batched])
    y_batched = y_test.values
    return x_batched_reshaped, y_batched

In [142]:
test_x_mod, test_y_mod = get_test_data()
test_y_predictions_probabilities = model.predict(test_x_mod)

In [143]:
print(len(test_y_predictions_probabilities))

8400


In [144]:
print(test_y_predictions_probabilities[0])

[9.9994004e-01 1.0670526e-06 4.7818770e-07 1.0533782e-09 9.7507723e-08
 8.5523479e-06 4.9100086e-05 6.1026134e-07 4.1489259e-10 1.1812699e-09]


In [146]:
test_y_predictions = [x.argmax() for x in test_y_predictions_probabilities]

In [147]:
print(len(test_y_predictions))

8400


In [148]:
print(test_y_predictions[0:10])

[0, 7, 7, 2, 2, 6, 5, 7, 8, 5]


In [149]:
print(test_y_mod[0:10])

[0 7 7 2 2 6 5 7 8 5]


In [151]:
correct_predictions = 0
total_count = len(test_y_mod)

for i in np.arange(total_count):
    if test_y_predictions[i] == test_y_mod[i]:
        correct_predictions = correct_predictions + 1

accuracy = correct_predictions / total_count
print('Accuracy: {}'.format(accuracy))

Accuracy: 0.9825
